In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
base_url = 'https://www.uefa.com/uefachampionsleague/season=2021/statistics/round=2001252/clubs/' #base URL
response = requests.get(base_url) #Send request
uefa_tables = pd.read_html(response.text)
df = uefa_tables[0] #Extract the dataframe from the list
teams=['Chelsea', 'Man. City']
df = df[df.Team.isin(teams)] #Filter by team
df_overview = df.sort_values(by='Team')
df_overview.index = ['Chelsea', 'Man. City']
df_overview

,Team,Matches played,Wins,Draws,Losses
Chelsea,Chelsea,12,8,3,1
Man. City,Man. City,12,11,1,0


In [3]:
def get_data(stat):
  stats_url = f'https://www.uefa.com/uefachampionsleague/season=2021/statistics/round=2001252/clubs/kind={stat}/index.html'
  response = requests.get(stats_url)
  uefa_tables = pd.read_html(response.text)
  df = uefa_tables[0]
  teams=['Chelsea', 'Man. City']
  df = df[df.Team.isin(teams)] #Filter by team
  return df.sort_values(by='Team')

In [4]:
stats_labels = ['goals', 'goaltypes', 'attempts', 'distribution', 'setpieces','disciplinary'] #labels for URL
df_stats = [df_overview]

for label in stats_labels:
  new_df = get_data(label)
  new_df.index = ['Chelsea', 'Man. City']
  df_stats.append(new_df)


In [5]:
df_champions = pd.concat(df_stats, axis=1) #Concat all DataFrames
df_champions

,Team,Matches played,Wins,Draws,Losses,Team,Total goals,Average scored,Total goals against,Average conceded,Goal difference,Team,Total goals,Left foot,Right foot,Header,Other,Own goals (for),Inside area,Outside area,Penalties,Team,Total attempts,Average per game,Attempts on target,Attempts off target,Attempts blocked,Attempts against woodwork,Team,PA,PC,PC %,Average ball possession (%),Average ball possession (time),Team,Total corners for,Average corners for,Total corners against,Average corners against,Crosses attempted,Crosses completed,Team,Matches played,Fouls committed,Fouls suffered,Yellow cards,Red cards
Chelsea,Chelsea,12,8,3,1,Chelsea,22,1.83,4,0.33,18,Chelsea,22,6,13,3,0,0,21,0,5,Chelsea,138,11.50,57,48,33,4,Chelsea,6751,5771,85%,52%,30 m,Chelsea,53,4.42,65,5.42,161,31,Chelsea,12,169,127,19,0
Man. City,Man. City,12,11,1,0,Man. City,25,2.08,4,0.33,21,Man. City,25,9,14,1,0,1,18,6,2,Man. City,163,13.58,72,54,37,3,Man. City,8524,7630,90%,58%,38 m,Man. City,70,5.83,25,2.08,198,38,Man. City,12,118,89,16,0


In [6]:
df_champions = df_champions.loc[:,~df_champions.columns.duplicated()]


In [7]:
path = '/content/drive/MyDrive/DataScience/RetoChampions/Data'
df_champions.to_csv(path+'/ChampionsLeague2021Stats.csv')

##Players

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
headers = {
  'sec-ch-ua': '" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Microsoft Edge\";v=\"90\"""',
  'Accept': '*/*',
  'Referer': 'https://www.uefa.com/uefachampionsleague/season=2021/statistics/round=2001252/players/index.html',
  'Prefer': 'safe',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua-mobile': '?0',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.62'
}

In [ ]:
url_overview = 'https://www.uefa.com/statistics/uefachampionsleague/season=2021/statistics/round=2001252/players/_loadRemaining.html' #base URL
response = requests.request("GET", url_overview, headers=headers) #Send request
response.encoding = response.apparent_encoding
player_tables = pd.read_html(response.text)
df_players_overview = player_tables[0]
df_players_overview.columns = ['Player', 'Matches Played', 'Minutes Played', 'Distance covered', 'Team']
teams=['Chelsea', 'Man. City']
df_players_overview = df_players_overview[df_players_overview.Team.isin(teams)] #Filter by team
df_players_overview = df_players_overview.sort_values(by='Player')
df_players_overview.info()



In [ ]:
df_players_overview.to_csv(path+'/PlayerOverview2021Stats.csv')

In [ ]:
def get_columns(stat):
  stats_url = f'https://www.uefa.com/uefachampionsleague/season=2021/statistics/round=2001252/players/kind={stat}/index.html'
  response = requests.get(stats_url)
  soup = BeautifulSoup(response.text, 'lxml')
  table = soup.table
  table_heads = table.find_all('th')
  column_names = []

  for colname in table_heads:
    column_names.append(str(colname.string))

  return column_names

In [ ]:
def get_players_data(stat):
  players_base_url = f'https://www.uefa.com/statistics/uefachampionsleague/season=2021/statistics/round=2001252/players/kind={stat}/_loadRemaining.html'
  response = requests.request("GET", players_base_url, headers=headers)
  response.encoding = response.apparent_encoding
  player_tables = pd.read_html(response.text)
  df_players = player_tables[0]
  column_names = get_columns(stat)
  df_players.columns = column_names
  teams=['Chelsea', 'Man. City']
  df_players = df_players[df_players.Team.isin(teams)] #Filter by team
  return df_players


In [ ]:
player_stats_labels = ['goals', 'goaltypes', 'attempts', 'distribution', 'disciplinary']
df_player_stats = []

for label in player_stats_labels:
  df_new = get_players_data(label)
  df_new = df_new.sort_values(by='Player')
  df_player_stats.append(df_new)


In [ ]:
df_player_goals = df_player_stats[0]
df_player_goals.info()


In [ ]:
df_player_goals.to_csv(path+'/PlayerGoals2021Stats.csv')

In [ ]:
df_player_goaltype = df_player_stats[1]
df_player_goaltype.info()

In [ ]:
df_player_goaltype.to_csv(path+'/PlayerGoalType2021Stats.csv')


In [ ]:
df_player_attempts = df_player_stats[2]
df_player_attempts.info()

In [ ]:
df_player_attempts.to_csv(path+'/PlayerAttempts2021Stats.csv')


In [ ]:
df_player_passes = df_player_stats[3]
df_player_passes.info()

In [ ]:
df_player_passes.to_csv(path+'/PlayerPasses2021Stats.csv')

In [ ]:
df_player_disciplinary = df_player_stats[4]
df_player_disciplinary.info()

In [ ]:
df_player_disciplinary.to_csv(path+'/PlayerDisciplinary2021Stats.csv')


In [ ]:
df_players = pd.concat(df_player_stats, axis=1)
df_players = df_players.loc[:,~df_players.columns.duplicated()]

In [ ]:
df_players.info()


#Cleaning

In [ ]:
df_players.isna().sum()

In [ ]:
df1 = df_players['Player'].dropna()

In [ ]:
df2 = pd.DataFrame(df1)
df2.info()

In [ ]:
players_array = np.array(df1)
players_array

In [ ]:
df_players = df_players[df_players.Player.isin(players_array)] #Filter by team
df_players

# FINALS


In [ ]:
finals_url = 'https://www.sportskeeda.com/football/champions-league-winners'
response = requests.get(finals_url)
uefa_tables = pd.read_html(response.text)
df = uefa_tables[0] #Extract the dataframe from the list
new_header = df.iloc[0] 
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [ ]:
df.to_csv(path+'/FinalsHistoryStats.csv')

#ODDS

In [ ]:
odds_url = 'https://www.vegasinsider.com/soccer/odds/champions-league/'
response = requests.get(odds_url)
soup = BeautifulSoup(response.text, 'lxml')
print(soup)
tables = soup.find_all('table')
#table_heads = table.find_all('th')
tables


In [ ]:
curl = 'https://sumo.com/api/site/37aef4c831d959ee1157949faa9141a171144c3a73bb8c789b3614d434e50a15/features?site_id=37aef4c831d959ee1157949faa9141a171144c3a73bb8c789b3614d434e50a15'

headers = {
  'authority': 'sumo.com',
  'sec-ch-ua': '" Not A;Brand\";v=\"99\", \"Chromium\";v=\"90\", \"Microsoft Edge\";v=\"90\"',
  'accept': 'application/json, text/plain, */*',
  'prefer': 'safe',
  'sec-ch-ua-mobile': '?0',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.62',
  'x-sumo-auth': 'z1PQgI9PHJToEN35GS6mPh2i',
  'origin': 'https://www.vegasinsider.com',
  'sec-fetch-site': 'cross-site',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': 'https://www.vegasinsider.com/',
  'accept-language': 'es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,pt;q=0.5',
  
}



In [ ]:
response = requests.request("GET", odds_url, headers=headers) #Send request
player_tables = pd.read_html(response.text)
df_players_overview = player_tables[0]
df_players_overview

In [ ]:
base_url = 'https://www.zamba.co/es/apuestas-deportivas/deportes/844/1351379392?gclid=CjwKCAjw47eFBhA9EiwAy8kzNI6cjdTU6q1D8h_cOMkTkyGvhxr9vynvsueUEzWFmsReeD-8jDI_EhoC1N8QAvD_BwE'
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'lxml')
#print(soup)
tables = soup.find_all('table')

tables

[]

In [ ]:
response = requests.get(base_url) #Send request
uefa_tables = pd.read_html(response.text)
df = uefa_tables[0] #Extract the dataframe from the list
df

ValueError: ignored